In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
import numpy as np
from household_package.data import call_data_url
from household_package.clean import clean_data
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from household_package.model import get_xy
import pandas as pd
from household_package.registry import save_model

In [3]:
df_raw = call_data_url()

In [4]:
percentiles = np.arange(0, 100, 0.25)
percentile_values = df_raw["KWH"].quantile(q=[p/100 for p in percentiles])
print(percentile_values)

0.0000       42.010000
0.0025      912.744250
0.0050     1185.774250
0.0075     1355.774625
0.0100     1506.111000
              ...     
0.9875    32491.116875
0.9900    33544.811500
0.9925    35389.482250
0.9950    38064.279250
0.9975    44079.164375
Name: KWH, Length: 400, dtype: float64


In [31]:

#percentile_min = df_raw['KWH'].quantile(0.005)
#percentile_max = df_raw['KWH'].quantile(0.995)

# Filter the DataFrame based on the specified percentiles
#filtered_df = df_raw[(df_raw['KWH'] >= percentile_min) & (df_raw['KWH'] <= percentile_max)]
#filtered_df

In [8]:
df = clean_data(df_raw)

In [18]:
X , y = get_xy(df)

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)

In [19]:
to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ',
                     'YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']
to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


preprocessor = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

model = LinearRegression()

pipe = Pipeline([('prep', preprocessor), ('model', model)])

In [45]:
model = pipe.fit(X_train, y_train)

In [46]:
model.score(X_test,y_test)

0.5055383972861784

In [31]:
model

SWIMPOOL                       0
NUMPORTEL                      0
SOLAR                          0
SMARTMETER                     0
REGIONC                     WEST
state_name            New Mexico
BA_climate             Mixed-Dry
TYPEHUQ          Single_detached
STORIES                        1
YEARMADERANGE              70-79
TOTROOMS                       8
WALLTYPE                   Brick
ROOFTYPE                 Shingle
WINDOWS                        4
NUMFRIG                        2
MICRO                          1
DISHWASH                       1
CWASHER                        1
DRYER                          1
TVCOLOR                        3
TELLWORK                       0
HEATHOME                       1
EQUIPM                   Furnace
AIRCOND                        1
NHSLDMEM                       2
SQFTEST                     2100
KWH                     12521.48
TOTAL_BATH                     3
TOTAL_COMP                     1
TOTAL_LIGHT                   17
Name: 0, d

In [34]:
X = df.drop('KWH',axis=1)

In [43]:
pd.DataFrame(X.iloc[0,:]).T

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,REGIONC,state_name,BA_climate,TYPEHUQ,STORIES,YEARMADERANGE,...,TVCOLOR,TELLWORK,HEATHOME,EQUIPM,AIRCOND,NHSLDMEM,SQFTEST,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,0,0,0,0,WEST,New Mexico,Mixed-Dry,Single_detached,1,70-79,...,3,0,1,Furnace,1,2,2100,3,1,17


In [44]:
pipe.predict(pd.DataFrame(X.iloc[0,:]).T)

array([10640.])

In [23]:
pipe.predict(df.iloc[0,:])

ValueError: X does not contain any features, but ColumnTransformer is expecting 29 features